In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report
import time
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
 
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from operator import truediv
import os
import scipy.io as sio
import math

In [2]:
def applyFA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    fa = FactorAnalysis(n_components=numComponents, random_state=0)
    newX = fa.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, fa
def loadData(name):
    data_path = os.path.join(os.getcwd(),'data')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
    
    return data, labels
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [3]:
## GLOBAL VARIABLES
dataset = 'PU'
test_ratio = 0.9775
windowSize = 12

In [4]:
X, y = loadData(dataset)
K = X.shape[2]
K = 3 if dataset == 'IP' else 3
X,fa = applyFA(X,numComponents=K)
X, y = createImageCubes(X, y, windowSize=windowSize)
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((962, 12, 12, 3), (41814, 12, 12, 3), (962,), (41814,))

In [5]:
# Convert numpy arrays to PyTorch tensors
Xtrain = torch.tensor(Xtrain, dtype=torch.float32).permute(0, 3, 1, 2)
Xtest = torch.tensor(Xtest, dtype=torch.float32).permute(0, 3, 1, 2)
ytrain = torch.tensor(ytrain, dtype=torch.long)
ytest = torch.tensor(ytest, dtype=torch.long)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

(torch.Size([962, 3, 12, 12]),
 torch.Size([41814, 3, 12, 12]),
 torch.Size([962]),
 torch.Size([41814]))

In [6]:
# Create DataLoaders
train_dataset = TensorDataset(Xtrain, ytrain)
test_dataset = TensorDataset(Xtest, ytest)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
import torch
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis, parameter_count

# Define your model
class PDCblock(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(PDCblock, self).__init__()
        self.growth_rate = growth_rate
        self.relu = nn.ReLU(inplace=True)

        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1_d1 = nn.Conv2d(in_planes, growth_rate, kernel_size=3, dilation=1, padding=1, bias=False)

        self.bn2_1 = nn.BatchNorm2d(in_planes)
        self.conv2_d1 = nn.Conv2d(in_planes, growth_rate, kernel_size=3, dilation=1, padding=1, bias=False)
        self.bn2_2 = nn.BatchNorm2d(growth_rate)
        self.conv2_d2 = nn.Conv2d(growth_rate, growth_rate, kernel_size=3, dilation=2, padding=2, bias=False)

        self.bn3_1 = nn.BatchNorm2d(in_planes)
        self.conv3_d1 = nn.Conv2d(in_planes, growth_rate, kernel_size=3, dilation=1, padding=1, bias=False)
        self.bn3_2 = nn.BatchNorm2d(growth_rate)
        self.conv3_d2 = nn.Conv2d(growth_rate, growth_rate, kernel_size=3, dilation=2, padding=2, bias=False)
        self.bn3_3 = nn.BatchNorm2d(growth_rate)
        self.conv3_d4 = nn.Conv2d(growth_rate, growth_rate, kernel_size=3, dilation=4, padding=4, bias=False)

    def forward(self, x):
        output_1 = self.conv1_d1(self.relu(self.bn1(x)))
        y1 = output_1

        output2_1 = self.conv2_d1(self.relu(self.bn2_1(x)))
        output2_2 = self.conv2_d2(self.relu(self.bn2_2(y1)))
        output2 = output2_1 + output2_2
        y2 = output2

        output3_1 = self.conv3_d1(self.relu(self.bn3_1(x)))
        output3_2 = self.conv3_d2(self.relu(self.bn3_2(y1)))
        output3_3 = self.conv3_d4(self.relu(self.bn3_3(y2)))
        output3 = output3_1 + output3_2 + output3_3
        y3 = output3
        output = torch.cat([y3, y2, y1, x], 1)
        return output

class Transition(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        return out

class PDCNet(nn.Module):
    def __init__(self, in_channels, num_classes, growth_rate=52, reduction=0.5):
        super(PDCNet, self).__init__()
        num_planes = 2*growth_rate
        self.growth_rate = growth_rate
        self.conv1 = nn.Conv2d(in_channels, num_planes, kernel_size=3, padding=1, bias=False)

        self.multiblock_1 = PDCblock(num_planes, growth_rate)
        num_planes += 3*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.multiblock_2 = PDCblock(num_planes, growth_rate)
        num_planes += 3*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.multiblock_5 = PDCblock(num_planes, growth_rate)
        num_planes += 3*growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes, num_classes)
        self.pooling = nn.AdaptiveAvgPool2d(1)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight = nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        out = x.squeeze(1)
        out = self.conv1(out)
        out = self.trans1(self.multiblock_1(out))
        out = self.trans2(self.multiblock_2(out))
        out = self.multiblock_5(out)
        out = self.pooling(F.relu(self.bn(out)))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [8]:

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PDCNet(in_channels=3, num_classes=len(set(ytrain.numpy())))
model = model.to(device)

# Summary
summary(model, input_size=(3, 12, 12))

# FLOPs and Parameters
dummy_input = torch.randn(1, 3, 12, 12).to(device)
flops = FlopCountAnalysis(model, dummy_input)
params = parameter_count(model)

print(f"FLOPs: {flops.total()}")
print(f"Parameters: {params['']}")


Unsupported operator aten::add_ encountered 21 time(s)
Unsupported operator aten::add encountered 9 time(s)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 104, 12, 12]           2,808
       BatchNorm2d-2          [-1, 104, 12, 12]             208
              ReLU-3          [-1, 104, 12, 12]               0
            Conv2d-4           [-1, 52, 12, 12]          48,672
       BatchNorm2d-5          [-1, 104, 12, 12]             208
              ReLU-6          [-1, 104, 12, 12]               0
            Conv2d-7           [-1, 52, 12, 12]          48,672
       BatchNorm2d-8           [-1, 52, 12, 12]             104
              ReLU-9           [-1, 52, 12, 12]               0
           Conv2d-10           [-1, 52, 12, 12]          24,336
      BatchNorm2d-11          [-1, 104, 12, 12]             208
             ReLU-12          [-1, 104, 12, 12]               0
           Conv2d-13           [-1, 52, 12, 12]          48,672
      BatchNorm2d-14           [-1, 52,

In [9]:
# Define model, criterion, optimizer
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = PDCNet(in_channels=3, num_classes=len(set(ytrain.numpy())))
#model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [10]:
# Training the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=25):
    model.train()
    start_time = time.time()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')
    end_time = time.time()
    training_time = end_time - start_time
    return training_time

# Evaluating the model
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    start_time = time.time()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    end_time = time.time()
    testing_time = end_time - start_time
    return all_labels, all_preds, testing_time

In [11]:
# Train the model
num_epochs = 50
training_time = train_model(model, train_loader, criterion, optimizer, num_epochs)

# Evaluate the model
y_true, y_pred, testing_time = evaluate_model(model, test_loader)

# Print classification report
print(classification_report(y_true, y_pred))

# Print training and testing time
print(f"Training time: {training_time:.2f} seconds")
print(f"Testing time: {testing_time:.2f} seconds")

Epoch 0/49, Loss: 1.0016
Epoch 1/49, Loss: 0.6949
Epoch 2/49, Loss: 0.6133
Epoch 3/49, Loss: 0.5775
Epoch 4/49, Loss: 0.4936
Epoch 5/49, Loss: 0.4680
Epoch 6/49, Loss: 0.5508
Epoch 7/49, Loss: 0.4163
Epoch 8/49, Loss: 0.3734
Epoch 9/49, Loss: 0.3263
Epoch 10/49, Loss: 0.4490
Epoch 11/49, Loss: 0.3755
Epoch 12/49, Loss: 0.2941
Epoch 13/49, Loss: 0.2369
Epoch 14/49, Loss: 0.2939
Epoch 15/49, Loss: 0.3068
Epoch 16/49, Loss: 0.2799
Epoch 17/49, Loss: 0.2517
Epoch 18/49, Loss: 0.2121
Epoch 19/49, Loss: 0.2603
Epoch 20/49, Loss: 0.2436
Epoch 21/49, Loss: 0.1898
Epoch 22/49, Loss: 0.2003
Epoch 23/49, Loss: 0.2166
Epoch 24/49, Loss: 0.2358
Epoch 25/49, Loss: 0.2100
Epoch 26/49, Loss: 0.1501
Epoch 27/49, Loss: 0.1999
Epoch 28/49, Loss: 0.1994
Epoch 29/49, Loss: 0.1653
Epoch 30/49, Loss: 0.0869
Epoch 31/49, Loss: 0.0768
Epoch 32/49, Loss: 0.1577
Epoch 33/49, Loss: 0.2774
Epoch 34/49, Loss: 0.2109
Epoch 35/49, Loss: 0.2044
Epoch 36/49, Loss: 0.1240
Epoch 37/49, Loss: 0.1098
Epoch 38/49, Loss: 0.1

In [12]:
# Overall Accuracy
oa = accuracy_score(y_true, y_pred)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
# Calculate per-class accuracy from the confusion matrix
class_accuracy = cm.diagonal() / cm.sum(axis=1)
# Average Accuracy
aa = np.mean(class_accuracy)

# Kappa Coefficient
kappa = cohen_kappa_score(y_true, y_pred)

print(f'Overall Accuracy (OA): {oa:.4f}')
print(f'Average Accuracy (AA): {aa:.4f}')
print(f'Kappa Coefficient: {kappa:.4f}')
for i, acc in enumerate(class_accuracy): print(f'Class {i+1} Accuracy: {acc:.4f}')

Overall Accuracy (OA): 0.9441
Average Accuracy (AA): 0.9164
Kappa Coefficient: 0.9262
Class 1 Accuracy: 0.9645
Class 2 Accuracy: 0.9638
Class 3 Accuracy: 0.5122
Class 4 Accuracy: 0.9679
Class 5 Accuracy: 0.9551
Class 6 Accuracy: 0.9616
Class 7 Accuracy: 0.9600
Class 8 Accuracy: 0.9942
Class 9 Accuracy: 0.9687
